<a href="https://colab.research.google.com/github/a-saadallah/geemap/blob/master/Detecting_Planting_and_Re_planting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Steps:

0. Establish the phenological profile/spectral signature of the region of interest generating a series time series line-graphs each showing the time series of the index value, labeled by value on the y axis, year on the x axis, and with each subplot title indicating the specific point being evaluated.*

1. Analyze the age of a perennial cropping system using the LANDSAT archive for a given point, generating an estimate of the sample point ages in a tabular format for each point.
--> use the Chen et al analysis for inspiration "Chen, Bin, Yufang Jin, and Patrick Brown. "Automatic mapping of planting year for tree crops with Landsat satellite time series stacks." ISPRS journal of photogrammetry and remote sensing 151 (2019): 176-188." 

2. Efficiently conduct the same analysis over a surface surrounding the point until a polygon boundary is reached, generating a **heatmap **of planting age and/or replanting for a given polygon boundary. 
(I'm working on getting the polygon boundaries for you -- for now we can start with a 100m buffer around each point)




### Importing modules & Authentication

In [3]:
# Import modules
import pandas as pd
import numpy as np
import ee
import folium
from google.colab import drive
import plotly.graph_objects as go

In [4]:
# Grant access to GDrive
# drive.mount('/content/drive')

In [6]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QPTOVA8kEjDdpaDsQOGd3P8yS1uebIA555F-upnzp3U&tc=wNIgLrxHt-BLASYFT5lcBlwltMXpxjUlLqaizUWXoQ0&cc=n9itN1LWzJr1_jfZSEZR77CSvIVfh9eTK1ibeGIvZkM

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWilaPiwWBFaPgzMOspzpTkK3Y6XhFQDSETsOT7ld9kX96L01NlQ2jU

Successfully saved authorization token.


# Loading Points txt file

If we want to load the points from a text file, then we can upload the text file to GDrive, get the file path, read it as Pandas data-frame. Another approach would be to copy the data from the original text file and cast them as lists. Both approaches are implemented here and you can uncomment (activate) the one you approve.

### Solution 1

In [ ]:
'''
# Reading txt file as pandas dataframe and then loop over the rows to 
# create a feature collection from the dataframe

# Path to the txt file in GDrive, the path after '/content/drive/My Drive/' is what 
# you need to change according to the loaction of the file in you drive
fileName = '/content/drive/My Drive/gee/starter25_v2.csv'
points_df = pd.read_csv(fileName)

# Empty GEE list
points_list = ee.List([])

# Filling the empty list with points features, each feature has a 'code' property
for point in range(len(points_df)):
    pointxyz = points_df.iloc[point]
    point_feat = ee.Feature(ee.Geometry.Point(
        [pointxyz[2], pointxyz[1]])).set('code', pointxyz[0])
    points_list = points_list.add(point_feat)

# Cast the filled list into a feature collection
points_FC = ee.FeatureCollection(points_list)

# Create buffer around points 
points_fc_buff = points_FC.map(lambda feat: 
                              feat.buffer(100))

'''

### Solution 2
This approach is functional but deactivated now. If you prefer this one, then remove the comments commas ''' located at the beginning and end of the code block, and deactivate the code block before. 


In [7]:
# Lists of txt file columns
code = [77980004, 208200001, 182140002, 205790001, 105970001, 128270001, 156690001, 184050001, 202030001, 70480001, 
        91810001, 104470001, 124660001,158830002, 173290002, 191570001, 210230001, 84820001, 197660001, 129860001, 
        143310001, 190920001, 124800003, 180750001, 188900001, 211050001]

lon = [-19.016389, -19.742372, -18.624723, -18.710832999999997, -18.755360999999997, -18.8365, -18.64075, 
       -18.800832999999997, -18.758611, -18.771473, -18.774723, -19.197091, -19.28005, -19.472948000000002, 
       -19.317223000000002, -19.508582999999998, -18.614722, -18.860556, -18.486775, -19.095833, -19.192277, 
       -19.115833, -19.026614000000002, -18.539444, -18.872445000000003, -18.516317]

lat = [-46.604444, -46.112403, -46.708889, -47.0625, -47.430917, -47.320917, -47.606694, -47.3425, -47.546389000000005, 
       -47.516444, -47.538889000000005, -46.145914000000005, -46.463761, -46.001639000000004, -46.128056, -46.058436, 
       -46.296389000000005, -45.859167, -46.666917, -46.624445, -46.526392, -46.604167, -47.920072999999995, -48.216111, 
       -48.001778, -48.467436]

# Empty GEE list
points_list = ee.List([])

# Filling the empty list with points features, each feature has a 'code' property
for point in range(len(code)):
    point_feat = ee.Feature(ee.Geometry.Point(
        [lat[point], lon[point]])).set('code', code[point])
    points_list = points_list.add(point_feat)

# Cast the filled list into a feature collection
points_FC = ee.FeatureCollection(points_list)

# Create buffer around points 
points_fc_buff = points_FC.map(lambda feat: feat.buffer(100))


# Computing NDVI

'cloudMaskL57' function masks clouds, cloud shadows, and computes NDVI for Landsat 5 & 7



In [8]:
def cloudMaskL57(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).Or(
        qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))

    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    masked_img = image.updateMask(cloud.Not())
    ndvi = masked_img.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return(ndvi.set('system:time_start', image.get('system:time_start')))

'cloudMaskL8' function masks clouds, cloud shadows, cirrus, and computes NDVI for Landsat-8 collection

In [9]:
def cloudMaskL8(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set or the cloud confidence (7) is high
    # or the cloud shadow bit is set (3) or the cirrus bit is set (9), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).Or(qa.bitwiseAnd(1 << 7)).Or(
        qa.bitwiseAnd(1 << 3)).Or(qa.bitwiseAnd(1 << 9))

    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    masked_img = image.updateMask(cloud.Not())
    ndvi = masked_img.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return(ndvi.set('system:time_start', image.get('system:time_start')))

Import Landsat series collections & filter each using polygons geometry




In [10]:
# Landsat 5 collection
landsat5_coll = ee.ImageCollection(
    'LANDSAT/LT05/C01/T1_SR').filterBounds(points_fc_buff)

# Landsat 7 collection
landsat7_coll = ee.ImageCollection(
    'LANDSAT/LE07/C01/T1_SR').filterBounds(points_fc_buff)

# Landsat 8 collection
landsat8_coll = ee.ImageCollection(
    'LANDSAT/LC08/C01/T1_SR').filterBounds(points_fc_buff)

Applying the mask & NDVI functions to collections, and then merge collections together into one image collection.

In [11]:
# For Landsat 5 collection
landsat5_ndvi = landsat5_coll.map(cloudMaskL57)

# For Landsat 7 collection
landsat7_ndvi = landsat7_coll.map(cloudMaskL57)

# For Landsat 8 collection
landsat8_ndvi = landsat8_coll.map(cloudMaskL8)

# Merge all collections together
landsat_578 = landsat5_ndvi.merge(landsat7_ndvi).merge(landsat8_ndvi)

# Retrieving Data
The annual statistical 'max' of the NDVI band is computed first on a pixel level. Within each annual max NDVI image, an extra band is created with a constant value of image year. This 'image year' band is of help when we turn the image collection into an array image(although the array should be sorted properly, sometimes while retrieving the data from GEE to python the sorting gets messed up).
 
 
*   0 axis will represent years
*   1 axis will have a list of two elements (the year max NDVI & the year number)




![Array Image Sample](https://drive.google.com/uc?id=19HiSPwWeYRKTQt9nM6yAXriDz_v626Dr)




### Annual NDVI maximum

In [12]:
# EE list object of years
years_seq_ls = ee.List.sequence(1984, 2019)

# NDVI Mask
ndvi_mask = landsat8_ndvi.filter(ee.Filter.calendarRange(2019, 2019, 'year')).max().select('NDVI').gt(0.8)


# Function that return image of two bands(annual max NDVI & year constant)
def annual_max(year):
    year_coll = landsat_578.filter(ee.Filter.calendarRange(year, year, 'year'))
    return year_coll.max().addBands(ee.Image.constant(ee.Number(year).toInt()).toFloat())


# Apply annual_max function and cast the output list into image collection then to array
yearly_coll = ee.ImageCollection(years_seq_ls.map(annual_max)).toArray().updateMask(ndvi_mask)


### Points map

In [13]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [14]:
# custom basemaps
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite Hybrid',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}


# Create a folium map object.
my_map = folium.Map(
    location=[-19.016389, -46.904444], zoom_start=8, height=800)

# Add custom basemaps
## comment the one you don't like
basemaps['Google Satellite Hybrid'].add_to(my_map)
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Terrain'].add_to(my_map)
basemaps['Google Satellite'].add_to(my_map)
#basemaps['Esri Satellite'].add_to(my_map)

# Add points with popup showing code number
## Turn points into datfarme
points_df = pd.DataFrame(zip(lon, lat, code), columns=['Lon', 'Lat', 'Code'])

## Loop over dataframe rows, add a marker with code popup
for index, row in points_df.iterrows():
    folium.Marker([row['Lon'], row['Lat']], popup=f'Poly_Code: {row["Code"].astype(int)}', icon=folium.Icon(
        icon='info-sign')).add_to(my_map)

# Add points buffer to the map object.
## In order to see the buffer, zoom in more!
folium.GeoJson(
    points_fc_buff.getInfo(),
    name='Points Buffer 100'
).add_to(my_map)

# Add the masked NDVI to the map object.
imageVisParam = {"opacity":0.5,"bands":["NDVI"],"gamma":1}
my_map.add_ee_layer(ndvi_mask, imageVisParam, 'General Mask Layer')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)

### Reducing per polygon feature
Reduce the array image using the polygons feature collection.
The pixels that fall under a polygon will be grouped as a list and added as a property (attribute) to the corresponding polygon
([Sample from GEE](https://drive.google.com/uc?id=1t0eazxjPwEkyqugbryC3rht2wl9KNV17))

In [15]:
ls_ser_val = yearly_coll.reduceRegions(
    collection= points_fc_buff, reducer=ee.Reducer.toList(), scale=30)

### Retrieve results to python

In this code block we retrieve the stored pixels lists (NDVI & Year) values from each polygon. We compute the mean and finally cast the results as Pandas dataframe. The final output of this code block is a list of length 26 (equal to the number of polygons) where each element is a dataframe of three columns (NDVI, Year, Polygon Code) and 36 rows (years from 1984 to 2019).

In [16]:
# Turn the polygons feature collection into a list to loop over each element
ls_ser_feat_list = ls_ser_val.toList(ls_ser_val.size())

# Empty list to be filled inside the next for loop
df_list = []

# Loop over each polygon, get the stored pixels values
## the result of each loop is a dataframe with annual NDVI, polygon code, and year columns
for feat in range(ls_ser_feat_list.size().getInfo()):
    feature = ls_ser_feat_list.get(feat)

    # The pixels list are stored under an attribute column called 'list'
    ls_feat = ee.Feature(feature).get('list').getInfo()

    # Turn the pixels array into numpy array
    arr = np.asarray(ls_feat)

    # Values that are 0 are turned into NA to avoid miscalculations while computing the mean
    arr[arr == 0] = np.nan

    # Computing the mean and creating a dataframe of mean values and years
    if len(arr) >= 2:
      feat_mean_df = pd.DataFrame(
          np.array(np.mean(arr, axis=(0))), columns=['NDVI', 'Year'])
    
      # Cast the years as integers (somehow it is stored as float when pulled from GEE)
      feat_mean_df['Year'] = feat_mean_df['Year'].astype(int)

      # Get polygon code & add it as a column to the dataframe
      feat_mean_df['Code'] = ee.Feature(feature).get('code').getInfo()
    else:
      continue

    # Adding the dataframe as an element to the empty list
    df_list.append(feat_mean_df)

# Plantation Year Detection

In this section, two approaches are developed and used to identify plantation year.

## First Approach
I designed this approach taking into consideration the nature of the planting/removal process and its influence on NDVI time series. The workflow can be broken down into the following:
 
 
*   Condition_1: Detection of valleys ( NDVI records located between two higher records)
*   Condition_2: Valleys below Z_Score (Z_Score is defined as one STDEV below mean)
*   Condition_3: Valleys where the NDVI difference to one or two previous records is higher than the difference of the following record. This is based on the fact that the removal of plantations causes a sudden drop in NDVI (observed in one to two years at max). On the contrary, it takes more than one year for a plantation to produce a healthy vegetative structure and hence it shows slow & gradual NDVI increase.
 
After applying the three previous conditions, one of three cases are produced:
 
 
1.   Only one record meets the conditions; it is considered to be the plantation record
2.   More than one record pass the conditions, then the most recent year is taken
3.   No records pass all the conditions (most certainly because no records are below Z_Score, which means that the polygon isn’t really covering plantations), we then apply only the first and last rule, and we take the lowest NDVI record.






In [17]:
# Empty data frame, will be filled by plantation year of each polygon
y_of_plant_self = pd.DataFrame()

# Loop over each polygon
for poly in range(len(df_list)):

  # polygon dataframe
  poly_data = df_list[poly].copy()

  # NDVI standard deviation
  std = poly_data['NDVI'].std()

  # NDVI mean
  mean = poly_data['NDVI'].mean()

  # standard deviation (Z-score)
  z_score = mean - std

  # Valleys
  poly_data['NDVI_after'] = poly_data['NDVI'].diff(-1)
  poly_data['NDVI_before'] = poly_data['NDVI'].diff(1)
  poly_data['NDVI_2_before'] = poly_data['NDVI'].diff(2)

  # Below Z_score
  below_zscore = poly_data['NDVI'] < z_score

  # Vallys_conditions
  valleys = (poly_data['NDVI_after'] < 0) & (poly_data['NDVI_before'] < 0)

  # increment controle condition
  bef_aft_incre = (poly_data['NDVI_before'] < poly_data['NDVI_after']) | (poly_data['NDVI_2_before'] < poly_data['NDVI_after'])

  # Applying rules
  years_of_plant = poly_data[valleys & bef_aft_incre & below_zscore]

  if len(years_of_plant) == 1:
    plant_year = years_of_plant[['Code', 'Year', 'NDVI']]
  elif len(years_of_plant) > 1:
    plant_year = years_of_plant.nlargest(1, 'Year')[['Code', 'Year', 'NDVI']]
  else:
    plant_year = poly_data[valleys & bef_aft_incre ].nsmallest(1, 'NDVI')[['Code', 'Year', 'NDVI']]
  y_of_plant_self = y_of_plant_self.append(plant_year, ignore_index= True)

## Second Approach
Chen developed an approach with 3 rules to estimate the planting year. The first rule is based on filtering NDVI years according to an adaptive Z-score. The initial Z-score threshold is computed as 2 standard deviations below the mean. If no year falls below that threshold, the Z-score magnitude is reduced iteratively by an interval of 0.2. The iteration is terminated when the upper NDVI threshold (0.4) is reached.
 
Rule no 2 is a decision rule designed to detect false plantation records. If the increment of NDVI values is higher than 0.4, then this record is considered a "false plantation". 
 
Rule no 3 aims at detecting the latest new plantations. It is based on selecting the most recent minimum plantation record.
 
 
---
 
 
 
Although the approach used by Chen et.al is robust, it is not scalable to other locations or types of plantations. This is mainly because he is using absolute thresholds and dilation values rather than relative ones in rules 1 & 2. In the following code block:
 
For Rule no 1
*   Z-score initial value is 2 standard deviation below mean
*   Z-score dilation magnitude is the range between the mean and 2 standard deviations divided by half number of values below mean
*   Z-score upper threshold is two-third the range between the mean and initial Z-score
 
For Rule no 2
*   The threshold for detection of a false plantation is computed as half the range between the mean and initial Z-score.
 






In [18]:
# Empty data frame, will be filled by plantation year of each polygon
y_of_plant_chen = pd.DataFrame()

# Loop over each polygon
for poly in range(len(df_list)):

  # polygon dataframe
  poly_data = df_list[poly]


  # NDVI standard deviation
  std = poly_data['NDVI'].std()

  # NDVI mean
  mean = poly_data['NDVI'].mean()

  # -2 standard deviation (Z-score)
  z_score = mean - 2*(std)

  # Add column of years NDVI increment
  ## This column shows the increment of NDVI from one year to the next
  below_mean = poly_data['NDVI'].diff(-1) * -1

  
  incre = (mean - z_score)/2
  poly_data['Rule_2'] = (below_mean < incre) & (below_mean > 0)


  # Valleys
  poly_data['NDVI_after'] = poly_data['NDVI'].diff(-1)
  poly_data['NDVI_before'] = poly_data['NDVI'].diff(1)

  # Vallys_conditions
  valleys = (poly_data['NDVI_after'] < 0) & (poly_data['NDVI_before'] < 0)

  # number of years where NDVI are below z_score
  val_below_zscore = poly_data[(poly_data['NDVI'] < z_score) & valleys]


    ###### Rule 1
  ## If no records below Zscore, Zscore should be increased by 0.2 magnetiude 
  ## while not exceding a threshold of 0.4. These values (from chen article) do not fit 
  ## the data here. Zscore threshold is set as one third of the range between Zscore & mean. The original Zscore will
  ## be increased by 0.03. At least 2 years records should be detected before reaching Zscore threshold.
  max_z_score = z_score + (mean - z_score)/3
  bel_mean_len = len(poly_data[poly_data['NDVI'] <= mean])
  ndvi_min = poly_data.min()['NDVI']
  z_score_dil = (mean - ndvi_min)/ bel_mean_len * 3

  while z_score <= max_z_score:
    if len(val_below_zscore) >= 3:
      val_below_zscore = val_below_zscore
      break
    else:
      z_score += z_score_dil
      val_below_zscore = poly_data[(poly_data['NDVI'] < z_score) & valleys]
      
  ### Rule no 2
  # Increment of NDVI one year after should be lower than 0.4
  rule_2 = val_below_zscore[val_below_zscore['Rule_2'] == True]

  if len(rule_2) == 1:
    plant_year = rule_2[['Code', 'Year', 'NDVI']]
  else:
    plant_year = val_below_zscore.nlargest(1, 'Year')[['Code', 'Year', 'NDVI']]

  y_of_plant_chen = y_of_plant_chen.append(plant_year, ignore_index= True)


## Plots

In [19]:
for ind in range(len(df_list)):
    poly_plot = df_list[ind].copy()
    y_of_plant_self_row = y_of_plant_self.iloc[ind]
    y_of_plant_chen_row = y_of_plant_chen.iloc[ind]
    std = poly_plot['NDVI'].std()
    poly_plot['mean'] = poly_plot['NDVI'].mean()
    poly_plot['std_p2'] = poly_plot['mean'] + 2*(std)
    poly_plot['std_m2'] = poly_plot['mean'] - 2*(std)
    poly_plot['std_m1'] = poly_plot['mean'] - std
    code = poly_plot['Code'][0]

    # Create traces
    fig = go.Figure()
    fig.layout.paper_bgcolor = "white"
    fig.layout.plot_bgcolor = "white"
    fig.update_xaxes(showline=True, linecolor="#444", showgrid=False,
                     ticks="outside", tickwidth=2, tickcolor='black', ticklen=5)
    fig.update_yaxes(showline=True, linecolor="#444", showgrid=False,
                     ticks="outside", tickwidth=2, tickcolor='black', ticklen=5)
    fig.update_layout(title=f'Fig. {ind + 1}: (Polygon Code: {code})')
    fig.add_trace(go.Scatter(
        x=poly_plot['Year'], y=poly_plot['NDVI'], mode='lines+markers', line_color='blue', name='NDVI'))
    fig.add_trace(go.Scatter(
        x=poly_plot['Year'], y=poly_plot['std_m2'], mode='lines', line_color='red', name='STDEV -2'))
    fig.add_trace(go.Scatter(
        x=poly_plot['Year'], y=poly_plot['std_m1'], mode='lines', line_color='black', name='STDEV -1'))
    fig.add_trace(go.Scatter(x=poly_plot['Year'], y=poly_plot['mean'], mode='lines', line=dict(
        color='green', width=2, dash='dash'), name='Mean'))
    # Create scatter trace of plantation year
    fig.add_trace(go.Scatter(x=[y_of_plant_chen_row[1]], y=[y_of_plant_chen_row[2]], text='Year of Plantation Chen_App', 
                             mode="markers", marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), 
                             name='Year of Plantation Chen_App'
                             ))
    
        # Create scatter trace of plantation year
    fig.add_trace(go.Scatter(x=[y_of_plant_self_row[1]], y=[y_of_plant_self_row[2]], text='Year of Plantation Our_App', 
                             mode="markers", marker=dict(size=12, line=dict(width=2, color='grey')), 
                             name='Year of Plantation Our_App'
                             ))

    fig.show()
